# player_parent_expertise
    This file combines player's life log data with their expertise and their parent's expertise.

In [2]:
%matplotlib inline

import os, re, glob, datetime, json
from os.path import join as opj
import pandas as pd
import numpy as np
import scipy.stats
import json

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

load family data

In [3]:
fam_data = pd.read_csv('../2_demographics/outputs/family_playerID.tsv', sep = '\t', index_col = 0)

/Users/dengwenning/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
fam_data.head()

,playerID,family
0,3080084,time-1592284232_eve-3080067_name-PICKLE
1,3080114,time-1592284232_eve-3080067_name-PICKLE
2,3080111,time-1592284232_eve-3080067_name-PICKLE
3,3080108,time-1592284232_eve-3080067_name-PICKLE
4,3080104,time-1592284232_eve-3080067_name-PICKLE


load player expertise data

In [8]:
player_exp = pd.read_csv('../2_demographics/outputs/player_expertise.tsv', sep = '\t')

In [10]:
player_exp = player_exp.loc[player_exp.era == 'boundless']

In [11]:
player_exp.head()

,era,timestamp,playerID,hash,age,n_life,gametime
7,boundless,1584163133,2783339.0,0002e5ea5ce7cfd761135d255a245a3344af4377,60.00,0,60.00
8,boundless,1584164596,2783430.0,0002e5ea5ce7cfd761135d255a245a3344af4377,23.77,1,83.77
9,boundless,1584199272,2784552.0,0002e5ea5ce7cfd761135d255a245a3344af4377,29.90,2,113.67
10,boundless,1584207170,2784982.0,0002e5ea5ce7cfd761135d255a245a3344af4377,1.43,3,115.10
11,boundless,1584207184,2784989.0,0002e5ea5ce7cfd761135d255a245a3344af4377,0.09,4,115.19


load life logs

In [16]:
life_logs = pd.read_csv('../2_demographics/outputs/all_lifelogs_compact.tsv', sep = '\t', index_col = [0])

/Users/dengwenning/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [17]:
life_logs.head()

,release,era,hash,playerID,tBirth,parent,birth,tDeath,death,age,cause_of_death,birthX,birthY,deathX,deathY,first,last
0,342.0,boundless,2cdc4d0b016f9c0d96d27013f1d19c42596e0c5c,3080130,1592288229,3080111,[-454640 41],1592288538,[-454625 49],5.14,hunger,-454640,41,-454625,49,CLAUDINE,P
1,342.0,boundless,a8b5975a81344f690f45ffc2554a0bc35af557a9,3080128,1592288191,3080105,[-454394 -4],1592291791,[-454383 15],60.00,oldAge,-454394,-4,-454383,15,DACY,LIGHTNING
2,342.0,boundless,c5838da39fd525a2ac24aee049dae6a42e502236,3080125,1592288138,3080101,[-311131 -322],1592291738,[-311112 -325],59.98,hunger,-311131,-322,-311112,-325,SHOKO,GREATHOUSE
3,342.0,boundless,11f9fce50bbc1ebbba5126434aea123e79249942,3080122,1592288029,3080104,[-454638 41],1592288696,[-454647 58],11.12,hunger,-454638,41,-454647,58,RANGER,PICKLE
4,342.0,boundless,b265b1958566b474094cf0282a0fa59e6d622885,3080120,1592287917,3080101,[-311111 -311],1592291365,[-311115 -66],57.46,hunger,-311111,-311,-311115,-66,HAPPY,GREATHOUSE


merge these 3 dfs

In [24]:
with_parent = pd.merge(player_exp, life_logs[['playerID', 'parent', 'age']])

In [27]:
with_parent = pd.merge(with_parent, fam_data)

In [28]:
with_parent.head()

,era,timestamp,playerID,hash,age,n_life,gametime,parent,family
0,boundless,1584163133,2783339.0,0002e5ea5ce7cfd761135d255a245a3344af4377,60.00,0,60.00,2783282,time-1584061498_eve-2779667_name-SLINKEY
1,boundless,1584164596,2783430.0,0002e5ea5ce7cfd761135d255a245a3344af4377,23.77,1,83.77,2783394,time-1584095324_eve-2781053_name-ARGENTINA
2,boundless,1584199272,2784552.0,0002e5ea5ce7cfd761135d255a245a3344af4377,29.90,2,113.67,2784466,time-1584176952_eve-2783860_name-EZEKIEL
3,boundless,1584210846,2784998.0,0002e5ea5ce7cfd761135d255a245a3344af4377,60.00,9,175.76,2784902,time-1584064999_eve-2780063_name-GOLD
4,boundless,1584210955,2785147.0,0002e5ea5ce7cfd761135d255a245a3344af4377,1.31,11,177.15,2785102,time-1584176952_eve-2783860_name-EZEKIEL


In [34]:
newdf = pd.merge(with_parent, player_exp[['playerID', 'gametime']], left_on='parent', right_on='playerID')

In [36]:
newdf = newdf.rename({'playerID_x': 'playerID', 'gametime_x': 'gametime', 'gametime_y': 'parent_gametime'}, axis = 1)
newdf = newdf[['timestamp', 'playerID', 'hash', 'age', 'n_life', 'gametime', 'parent', 'parent_gametime', 'family']]

In [37]:
newdf.head()

,timestamp,playerID,hash,age,n_life,gametime,parent,parent_gametime,family
0,1584163133,2783339.0,0002e5ea5ce7cfd761135d255a245a3344af4377,60.00,0,60.00,2783282,2251.49,time-1584061498_eve-2779667_name-SLINKEY
1,1584163415,2783340.0,2eadd6f0018a6d6c02f98d1c411ba5e2acc3f7c9,60.00,35,876.03,2783282,2251.49,time-1584061498_eve-2779667_name-SLINKEY
2,1584163498,2783342.0,36cccaa18396001a641ed8739b26b7cbea88e4ff,59.25,504,14469.71,2783282,2251.49,time-1584061498_eve-2779667_name-SLINKEY
3,1584164596,2783430.0,0002e5ea5ce7cfd761135d255a245a3344af4377,23.77,1,83.77,2783394,19084.51,time-1584095324_eve-2781053_name-ARGENTINA
4,1584162947,2783417.0,2c117e3ea7cbe831a2fa4abb4ffb83a643d55baa,0.99,5,68.87,2783394,19084.51,time-1584095324_eve-2781053_name-ARGENTINA


check is lengths change

In [29]:
len(with_parent)

329510

In [30]:
len(fam_data)

1318098

In [21]:
len(player_exp)

775069

In [22]:
len(life_logs)

1671032

In [38]:
len(newdf)

321513

save to csv (don't save for now as the lengths are not same)

In [ ]:
# newdf.to_csv('../outputs/player_parent_expertise.csv')